In [ ]:
# pip uninstall -y trulens_eval
# pip install git+https://github.com/truera/trulens@piotrm/azure_bugfixes#subdirectory=trulens_eval

# trulens_eval notebook dev

%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

base = Path().cwd()
while not (base / "trulens_eval").exists():
    base = base.parent

print(base)

# If running from github repo, can use this:
sys.path.append(str(base))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

# from trulens_eval import Tru
# tru = Tru()
# tru.reset_database()
# tru.run_dashboard(_dev=base, force=True)

In [ ]:
from trulens_eval.feedback.v2.feedback import Template, Insensitivity

t = Template.from_template("hello there")

In [ ]:
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval import Select
from trulens_eval.feedback.feedback import Feedback

In [ ]:
import inspect
from typing import Tuple
from trulens_eval.utils.text import make_retab

fimp = Dummy().language_match

invert_template = """
You are a fuzzing tool. Your job is to provide inputs to a function that will
achieve a desired output. You are given a description of the function. Some
inputs may be fixed, others are to be determined by you.

BEGIN FUNCTION DESCRIPTION
{imp_doc}
END FUNCTION DESCRIPTION

BEGIN TARGET OUTPUT
{target}
END TARGET OUTPUT

BEGIN FIXED INPUTS
{fixed_inputs}
END FIXED INPUTS
"""

def invert_feedback(imp, target, **kwargs):
    """Try to fill in input values to feedback function `imp` to achieve output
    value `target`. Any `kwargs` provided fix `imp` arguments to the given
    values.
    """

    doc = imp.__doc__

    filled_template = invert_template.format(
        imp_doc=doc,
        target=target,
        fixed_inputs="\n".join(
            f"{k}={v}" for k, v in kwargs.items()
        )
    )

    return filled_template

#BEGIN FUNCTION DOCSTRING
#{imp_doc}
#END FUNCTION DOCSTRING

doc_template = """
You are a python method summarization tool. Your job is to summarize the
purpose, implementation, arguments, and returns of a given method based on its
signature and source code.

BEGIN FUNCTION SIGNATURE
{sig}
END FUNCTION SIGNATURE

BEGIN FUNCTION SOURCE
{src}
END FUNCTION SOURCE

Summarize the purpose of the method. Please be as concise as possible and avoid
mentioning how the method is implemented or what tools it uses to achieve its
purpose. Only summarize the purpose.

PURPOSE: <overall method purpose>

Summarize how the method is implemented:
IMPLEMENTATION: <how the method achieves its purpose>

Summarize each of these arguments in this form, one per line:
{args_templates}

Summarize the function's return value. List its type, overall interpretation,
and an interpretation extremal values it could achieve.
{rets_templates}
"""

arg_template = """ARGUMENT({name}: {type}): <argument_description>"""

ret_template = """
RETURN({type}): <return1_description>
RETURNVALUE(<return1_value1>): <interpretation for this return value>
RETURNVALUE(<return1_value2>): <interpretation for this return value>
"""

def doc_feedback(imp):
    """Try to fill in input values to feedback function `imp` to achieve output
    value `target`. Any `kwargs` provided fix `imp` arguments to the given
    values.
    """

    doc = imp.__doc__
    sig = inspect.signature(imp)

    rt = make_retab("  ")

    arg_templates = "\n".join(
        arg_template.format(name=arg.name, type=arg.annotation.__name__)
        for arg in sig.parameters.values() if arg.name != "self"
    )

    ret_annot = sig.return_annotation

    if hasattr(ret_annot, "__args__"):
        ret_types = [ret_annot.__args__[0]]
    else:
        ret_types = [ret_annot]

    rets_templates = "\n".join(
        ret_template.format(type=ret_type.__name__)
        for ret_type in ret_types
    )
    
    filled_template = doc_template.format(
        imp_doc=rt(doc),
        sig=rt(imp.__name__ + str(sig)),
        src=rt(inspect.getsource(fimp)),
        args_templates=arg_templates,
        rets_templates=rets_templates
    )

    return filled_template


# invert_feedback(fimp, 1.0, text1="How are you?")
prompt = doc_feedback(Dummy().toxic)

In [ ]:
import dataclasses
from typing import Dict, Any, Type
import re
from textwrap import wrap

import docstring_parser

@dataclasses.dataclass
class LLMDoc():
    purpose: str

    details: str

    args: Dict[Tuple[str, Type], str]

    ret: str
    ret_type: Type
    rets: Dict[Any, str]

    other: Dict[str, str] = dataclasses.field(default_factory=dict)

    @classmethod
    def of_docstring(cls, doc):
        ds = docstring_parser.parse(doc)
        
        return cls(
            purpose = ds.short_description,
            details = ds.long_description,
            args = {
                (arg.arg_name, arg.type_name): arg.description
                for arg in ds.params
            },
            ret = ds.returns.description,
            ret_type = ds.returns.type_name,
            rets = {},
            # rets = {
            #    ret.arg_name: ret.description
            #    for ret in ds.returns.returns
            #}
        )

    def as_docstring(self, initial_indent=0, tabsize=4):
        tab1 = make_retab("  ")

        space = 80 - initial_indent

        doc_purpose = "\n".join(wrap(
            "\"\"\"" + self.purpose,
            width=space,
            initial_indent=" " * initial_indent,
            subsequent_indent=" " * initial_indent
        ))

        doc_details = "\n".join(wrap(
            self.details,
            width=space,
            initial_indent=" " * initial_indent,
            subsequent_indent=" " * initial_indent
        ))

        doc_args = " " * initial_indent + "Args:\n"
        for (name, type_), desc in self.args.items():
            doc_args += "\n".join(wrap(
                f"{name}: {desc}",
                width=space - initial_indent - tabsize,
                initial_indent=" " * (initial_indent + tabsize),
                subsequent_indent=" " * (initial_indent + 2*tabsize)
            )) + "\n\n"

        doc_rets = " " * initial_indent + "Returns:\n"
        doc_rets += "\n".join(wrap(
            f"{self.ret_type}: {self.ret}",
            width=space - tabsize,
            initial_indent=" " * (initial_indent + tabsize),
            subsequent_indent=" " * (initial_indent + 2*tabsize)
        )) + "\n\n"

        for ret, desc in self.rets.items():
            doc_rets += "\n".join(wrap(
                f"{ret}: {desc}",
                width=space - 2*tabsize,
                initial_indent=" " * (initial_indent + 2*tabsize),
                subsequent_indent=" " * (initial_indent + 3*tabsize)
            )) + "\n\n"

        
        doc = f"""{doc_purpose}

{doc_details}

{doc_args}

{doc_rets}
{" " * initial_indent}\"\"\"
"""
        return doc

re_purpose = re.compile(r"PURPOSE: (.*)")
re_arg = re.compile(r"ARGUMENT\((.*): (.*)\): (.*)")
re_returnvalue = re.compile(r"RETURNVALUE\((.*)\): (.*)")
re_return = re.compile(r"RETURN\((.*)\): (.*)")
re_details = re.compile(r"DETAILS: (.*)")

def parse_result(res: str) -> LLMDoc:
    lines = res.split("\n")

    doc_args = {}
    doc_rets = {}
    doc = {'args': doc_args, 'rets': doc_rets}

    for line in lines:
        if match := re_purpose.fullmatch(line):
            doc['purpose'] = match.group(1)

        elif match := re_details.fullmatch(line):
            doc['details'] = match.group(1)

        elif match := re_returnvalue.fullmatch(line):
            arg, desc = match.groups()
            doc_rets[arg] = desc

        elif match := re_return.fullmatch(line):
            doc['ret'] = match.group(2)
            doc['ret_type'] = match.group(1)

        elif match := re_arg.fullmatch(line):
            name, type_, desc = match.groups()
            doc_args[(name, type_)] = desc

    return LLMDoc(**doc)

from pprint import pprint

doc = parse_result("""PURPOSE: Calculate the likelihood that two texts are in the same natural language.

DETAILS: The method uses a language detection API to get language scores for the input texts, calculates the difference in scores, and returns a value based on the difference.

ARGUMENT(text1: str): The first text to evaluate.
ARGUMENT(text2: str): The second text to evaluate.

RETURN(float): A value between 0.0 and 1.0. The value 0.0 indicates that the input texts were of different languages, and 1.0 indicates they are in the same language.
RETURNVALUE(0.0): Input texts are of different languages.
RETURNVALUE(1.0): Input texts are in the same language.""")

print(doc.as_docstring(initial_indent=4))

In [ ]:
# res = c.completions.create#(model="gpt-3.5-turbo", prompt=prompt)
# help(c.chat.completions.create)

import openai
c = openai.OpenAI()

d = Dummy()

for fimp in [d.positive_sentiment]:#, d.positive_sentiment, d.toxic, d.pii_detection, d.hallucination_evaluator]:
    print(fimp.__name__)
    prompt = doc_feedback(fimp)
    res = c.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "system", "content": prompt}], temperature=0.0)
    print(res.choices[0].message.content)
    print()

In [ ]:
from typing import Optional, Set, List
import inspect
from ipywidgets import widgets
from inspect import isclass, ismodule, isfunction, ismethod, isabstract
import os

MD = 4

def pypackage(package: str, filter) -> List[Tuple[Any,...]]:
    added = set()

    mod = __import__(package)
    pkgpath = os.path.dirname(mod.__file__)

    rets = []

    import pkgutil
    for _, name, _ in pkgutil.iter_modules([pkgpath]):
        try:
            mod = __import__(package + "." + name)
            subrets = pytree(obj=mod, package=package, added=added, depth=MD, filter=filter)
            if subrets:
                rets.append((
                    0,
                    package + "." + name,
                    type(mod).__name__,
                    mod
                ))
                for subret in subrets:
                    rets.append((
                        subret[0]+1,
                        *subret[1:]
                    ))

        except ImportError:
            pass

    return rets

def pytree(obj, package: str = "trulens_eval", added: Optional[Set[int]] = None, depth: int = MD, filter=None) -> List[Tuple[Any,...]]:
    """Create a tree of the module's contents."""

    try:
        pack = inspect.getattr_static(obj, "__package__")
        if pack != package:
            return []
        
    except AttributeError:
        pass

    if depth == 0:
        return []

    if added is None:
        added = set()

    if id(obj) in added:
        return []
    
    added.add(id(obj))

    rets = []

    if filter is None:
        f = lambda _: True
        
    elif isinstance(filter, str):
        def f(obj):
            cls = type(obj)
            content = str(obj)
            cls_name = cls.__name__
            doc = obj.__doc__
            return eval(filter)
            
    for name, subobj in inspect.getmembers_static(obj):
        if id(subobj) in added:
            continue

        subpack = None
        try:
            subpack = inspect.getattr_static(subobj, "__package__")
            subname = subpack + "." + name

        except AttributeError:
            subname = name
            pass

        if subpack is not None and subpack != package:
            continue

        if ismodule(subobj):
            subrets = pytree(subobj, package, added, depth-1, filter)
            if subrets or f(subobj):
                rets.append((
                    MD-depth,
                    subname,
                    str(type(subobj).__name__),
                    subobj,
                ))
                for subret in subrets:
                    rets.append(subret)

        elif inspect.isclass(subobj):
            subrets = pytree(subobj, package, added, depth-1, filter)
            if subrets or f(subobj):
                rets.append((
                    MD-depth,
                    subname,
                    str(type(subobj).__name__),
                    subobj
                ))
                for subret in subrets:
                    rets.append(subret)

        elif isinstance(subobj, (classmethod, staticmethod)):
            base = obj.__class__.__base__

            if hasattr(base, name):
                continue
            
            if f(name):
                rets.append((
                    MD-depth,
                    subname,
                    str(type(subobj).__name__),
                    subobj
                ))

        else:
            continue

    return rets

In [ ]:
import trulens_eval

In [ ]:
from ipywidgets import interact
from ipywidgets import widgets
import ipyregulartable as rt
import pandas as pd

w_count_progress = widgets.IntProgress(min=0, max=0, value=0)
w_count_label = widgets.Label("")
w_filter = widgets.Text()
w_error = widgets.Output()
w_output = widgets.Output()

display(w_filter)
display(widgets.HBox([w_count_progress, w_count_label]))
display(w_error)
display(w_output)

w_rows = []

max_rows = 20

@dataclasses.dataclass
class UIRow():
    obj: Any

    w_select: widgets.Button
    w_name: widgets.Label
    w_type: widgets.Label
    w_doc: widgets.Label

    def set_obj(self, obj, name: Optional[str] = None):
        self.obj = obj
        if name is None:
            if "__name__" in dir(obj):
                name = obj.__name__
            else:
                name = str(obj)
        self.w_name.value = name
        self.w_type.value = type(obj).__name__
        self.w_doc.value = "0" if obj.__doc__ is None else str(len(obj.__doc__))

    def __init__(self, raw):
        self.raw = raw
        self.w_select = widgets.Button(description="Select")
        self.w_name = widgets.Label()
        self.w_type = widgets.Label()
        self.w_doc = widgets.Label()

    def render(self):
        return widgets.HBox([self.w_select, self.w_name, self.w_type, self.w_doc])

for i in range(max_rows):
    w_rows.append(
        UIRow(raw=(None, None, None))
    )

with w_output:
    for i, row in enumerate(w_rows):
        display(row.render())

def browse(ev: str):
    filter = ev.new

    obj = trulens_eval
    content = str(obj)
    cls = type(trulens_eval)
    cls_name = cls.__name__
    doc = obj.__doc__
    
    try:
        test = eval(filter)
        if not isinstance(test, bool):
            raise ValueError("filter needs to be a boolean expression")

        rows = pypackage("trulens_eval", filter=filter)

        n_rows = len(rows)
        if n_rows > w_count_progress.max:
            w_count_progress.max = n_rows
        
        w_count_progress.value = n_rows
        w_count_label.value = f"{n_rows}/{w_count_progress.max} {100*n_rows/w_count_progress.max:0.2f}%"

        for i, row in enumerate(rows):
            if i >= max_rows:
                break
            w_rows[i].set_obj(row[3], name=row[1])

        w_error.clear_output()
        
    except Exception as e:
        w_error.clear_output()
        with w_error:
            print(e)

w_filter.observe(browse, names="value")
w_filter.value = "True"

In [ ]:
! pip install plotly

In [ ]:
from typing import Optional, Set, List
import inspect
from ipywidgets import widgets
from inspect import isclass, ismodule, isfunction, ismethod, isabstract
import os
from importlib import import_module
import pkgutil

MD = 4

def qualname(obj):
    parts = []

    if hasattr(obj, "__module__") and isinstance(obj.__module__, str):
        parts.append(obj.__module__)
    
    if hasattr(obj, "__qualname__") and isinstance(obj.__qualname__, str):
        parts.append(obj.__qualname__)

    else:
        try:
            name = getattr(obj, "__name__")

            if isinstance(name, str):
                parts.append(name)

        except AttributeError:
            pass
    
    return ".".join(parts)

@dataclasses.dataclass
class RowRel():
    obj: Any

    depth: int
    label: str
    parent: str
    
def pypackage_rel(
    package_name: str,
    filter,
    added: Optional[Set[int]] = None,
    added_paths: Optional[Set[str]] = None,
) -> List[RowRel]:

    if added is None:
        added = set()

    if added_paths is None:
        added_paths = set()

    try:
        mod = import_module(package_name)
    except Exception:
        return []
    
    path = mod.__file__
    pkgpath = os.path.dirname(path)

    if path in added_paths:
        return []

    added_paths.add(path)
    
    rets = []

    # print(package_name, mod, mod.__file__, pkgpath)

    parts = package_name.split(".")
    if len(parts) > 1:
        parent_package_name = ".".join(parts[0:-1])
    else:
        parent_package_name = ""

    print("adding", package_name, parent_package_name, path)

    rets.append(RowRel(obj=mod, depth=0, label=package_name, parent=parent_package_name))
    print("subrets call", mod, package_name, path)
    subrets = pytree_rel(obj=mod, parent_qualname=parent_package_name, added=added, depth=MD, filter=filter, path=path)
    
    for subret in subrets:
        print("want to add subret", subret.label, subret.parent, subret.depth)
    #    rets.append(subret)

    if pkgpath in added_paths:
        return rets
    
    added_paths.add(pkgpath)
    
    for i, (_, name, _) in enumerate(pkgutil.iter_modules([pkgpath])):
        if i > 2:
            break
        
        sub_package_name = package_name + "." + name

        print("\t", package_name, sub_package_name)

        try:
            submod = import_module(sub_package_name)
            if submod.__file__ in added_paths:
                continue

        except Exception:
            continue    

        #subrets = pytree_rel(obj=mod, package=package_name, added=added, depth=MD, filter=filter)
        #if subrets:
        # rets.append(RowRel(obj=mod, depth=0, label=package_name + "." + name, parent=package_name))
        # rets.append(RowRel(obj=submod, depth=1, label=sub_package_name, parent=package_name))

        subrets = pypackage_rel(package_name=sub_package_name, added=added, filter=filter, added_paths=added_paths)
        
        for subret in subrets:
            subret.depth += 1
            rets.append(subret)
            
        #except Exception:
        #    pass


    return rets

def pytree_rel(
    obj: Any,
    path: str,
    parent_qualname: str = "trulens_eval",
    added: Optional[Set[int]] = None,
    depth: int = MD,
    filter=None,
) -> List[RowRel]:
    """Create a tree of the module's contents."""

    if depth == 0:
        return []

    try:
        obj_qualname = qualname(obj)

    except Exception:
        return []

    if ismodule(obj):    
        if obj.__file__ != path:
            print("SKIP due to obj.__file__", obj.__file__, path)
            return []
        
    elif isclass(obj):
        if not obj.__module__.startswith(parent_qualname):
            print("SKIP due to obj.__module__", obj.__module__, parent_qualname)
            return []
        
    if ismethod(obj) or isfunction(obj):
        if not obj.__module__.startswith(parent_qualname):
            print("SKIP due to obj.__module__", obj.__module__, parent_qualname)
            return []

    if not obj_qualname.startswith(parent_qualname):
        return []

    if added is None:
        added = set()

    if id(obj) in added:
        return []
    
    added.add(id(obj))

    rets = []
    
    if filter is None:
        f = lambda _: True
        
    elif isinstance(filter, str):
        def f(obj):
            cls = type(obj)
            content = str(obj)
            cls_name = cls.__name__
            doc = obj.__doc__
            return eval(filter)
    
    rets.append(RowRel(obj=obj, depth=MD-depth, label=obj_qualname, parent=parent_qualname))

    for attr_name, subobj in inspect.getmembers_static(obj):
        if id(subobj) in added:
            continue

        try:
            subname = qualname(subobj)
        except Exception:
            continue

        if not subname.startswith(parent_qualname):
            continue

        if hasattr(subobj, "__file__"):
            if subobj.__file__ != path:
                print("SKIP due to subobj.__file__", subobj.__file__, path)
                continue

        if ismodule(subobj):
            continue
            subrets = pytree_rel(subobj, parent_qualname=parent_qualname, added=added, depth=depth-1, filter=filter, path=path)
            if subrets or f(subobj):
                rets.append(RowRel(obj=subobj, depth=MD-depth, label=subname, parent=obj_qualname))
                for subret in subrets:
                    rets.append(subret)

        # print("consider", attr_name, "under", name, "qualname=", subname)

        if inspect.isclass(subobj):
            if not subname.startswith(obj_qualname):
                continue

            rets.append(RowRel(obj=subobj, depth=MD-depth, label=subname, parent=obj_qualname))

            continue

            subrets = pytree_rel(subobj, parent_qualname=subname, added=added, depth=depth-1, filter=filter, path=path)
            if subrets or f(subobj):
                rets.append(RowRel(obj=subobj, depth=MD-depth, label=subname, parent=obj_qualname))
                for subret in subrets:
                    rets.append(subret)

        elif isinstance(subobj, (classmethod, staticmethod)):
            bases = obj.__bases__

            if any(hasattr(base, attr_name) for base in bases):
                continue
            
            if f(subobj):
                rets.append(RowRel(obj=subobj, depth=MD-depth, label=subname, parent=obj_qualname))

        elif isfunction(subobj):
            if f(subobj):
                rets.append(RowRel(obj=subobj, depth=MD-depth, label=subname, parent=obj_qualname))
        
        else:
            # print(str(subobj))
            continue

    return rets

In [ ]:
import plotly.graph_objects as go

rows = pypackage_rel("trulens_eval", filter="True")[0:100]

"""
fig = go.Figure(go.Treemap(
    labels = [row.label for row in rows],
    parents = [row.parent for row in rows],
    # root_color="lightgrey"
))

fig
"""

In [ ]:
[(row.label, row.parent) for row in rows][0:100]

In [ ]:
from trulens_eval.feedback.provider.openai import AzureOpenAI
qualname(AzureOpenAI)

In [ ]:
temp = import_module("trulens_eval.Example_TruBot")
temp.__file__

In [ ]:
# ! pip install nemoguardrails

In [ ]:
from typing import Optional, Set, List, Iterable
import inspect
from ipywidgets import widgets
from inspect import isclass, ismodule, isfunction, ismethod, isabstract
import os
from importlib import import_module
import pkgutil

MD = 4

def is_direct_sub(subqual, parentqual):
    if parentqual == "":
        parentparts = []
    else:
        parentparts = parentqual.split(".")

    if subqual == "":
        subparts = []
    else:
        subparts = subqual.split(".")
    
    return subparts[0:-1] == parentparts

def qualname(obj):
    parts = []

    if hasattr(obj, "__module__") and isinstance(obj.__module__, str):
        parts.append(obj.__module__)
    
    if hasattr(obj, "__qualname__") and isinstance(obj.__qualname__, str):
        parts.append(obj.__qualname__)

    else:
        try:
            name = getattr(obj, "__name__")

            if isinstance(name, str):
                parts.append(name)

        except AttributeError:
            pass
    
    return ".".join(parts)

@dataclasses.dataclass
class Visit():
    obj: Any

    typ: str

    depth: int

    # docstring: str

    qualname: str

    parent: Any
    
def visit(
    obj: Any,
    added_objects: Optional[Set[int]] = None,
    added_paths: Optional[Set[str]] = None,
    parent_qualname: str = "",
    parent_path: Optional[str] = None,
    depth: int = 0,
) -> Iterable[Visit]:

    if added_objects is None:
        added_objects = set()

    if added_paths is None:
        added_paths = set()

    if id(obj) in added_objects:
        return

    if ismodule(obj):
        if not hasattr(obj, "__file__"):
            return

        object_path = obj.__file__
        
        if object_path is None:
            module_path = None
        else:
            module_path = os.path.dirname(object_path)

        if object_path is not None and object_path in added_paths:
            print("skip", object_path)
            return

    elif isclass(obj):
        #if not hasattr(obj, "__module__") or obj.__module__ != parent_qualname:
        #    return
        #if not hasattr(obj, "__module__") or obj.__module__ != parent_qualname:
        #    return
        pass

    elif ismethod(obj) or isfunction(obj):
        #if not hasattr(obj, "__module__") or obj.__module__ != parent_qualname:
        #    return
        pass

    else:
        return

    obj_qualname = qualname(obj)
    obj_qualname_parts = obj_qualname.split(".")
    parent_qualname_parts = parent_qualname.split(".")

    # if parent_qualname and (not obj_qualname_parts[0:-1] == parent_qualname_parts):
    if not is_direct_sub(obj_qualname, parent_qualname):
        print("skip", obj_qualname, parent_qualname)
        return

    added_objects.add(id(obj))

    if ismodule(obj):
        added_paths.add(object_path)

    yield Visit(
        obj=obj,
        typ=type(obj).__name__,
        depth=depth,
        qualname=obj_qualname,
        parent=parent_qualname
    )

    for _, subobj in inspect.getmembers_static(obj):
        yield from visit(
            obj=subobj,
            added_objects=added_objects,
            added_paths=added_paths,
            parent_qualname=obj_qualname,
            depth=depth+1
        )

    # The above will not visit anything in a module that has __init__.py that is
    # not defined in the __init__.py file. The below will visit all submodules
    # based on their location in the filesystem.
    if ismodule(obj) and obj.__file__ is not None:
        if not obj.__file__.endswith("__init__.py"):
            return

        mod_folder = os.path.dirname(obj.__file__)
        for i, (_, modname, _) in enumerate(pkgutil.iter_modules([mod_folder])):
            subqualname = obj_qualname + "." + modname

            try:
                submod = import_module(subqualname)

                yield from visit(
                    obj=submod,
                    added_objects=added_objects,
                    added_paths=added_paths,
                    parent_qualname=obj_qualname,
                    depth=depth+1
                )
            except Exception:
                pass

            # print(obj_qualname, name)

    return

    # subrets = pytree_rel(obj=mod, parent_qualname=parent_package_name, added=added, depth=MD, filter=filter, path=path)


In [ ]:
trulens_eval.tru_chain.__file__

In [ ]:
is_direct_sub("trulens_eval", "")

In [ ]:
visits = list(visit(trulens_eval))

qualnames = [v.qualname for v in visits]
parents = [v.parent for v in visits]
depths = [v.depth for v in visits]

def desc(obj):
    doc = obj.__doc__

    if doc is None:
        return ""
    
    temp = docstring_parser.parse(doc)

    if temp.short_description is not None:
        return temp.short_description
    
    return "None"

def doc_goodness(obj):
    if isclass(obj):
        return "gray"

    if not (isfunction(obj) or ismethod(obj)):
        return "black"
    
    if obj.__doc__ is None:
        return "red"

    temp = docstring_parser.parse(obj.__doc__)

    if len(temp.short_description) > 50:
        return "orange"
    
    return "green"


docstrings = [desc(v.obj) for v in visits]
goodness = [doc_goodness(v.obj) for v in visits]

In [ ]:
import plotly.graph_objects as go

# rows = pypackage_rel("trulens_eval", filter="True")[0:100]

fig = go.Figure(go.Treemap(
    labels = qualnames,
    parents = parents,
    text = [docstring for docstring in docstrings],
    marker_colors = goodness,
    # root_color="green"
))

fig


In [ ]:
list(zip(qualnames, parents))[0:1000]

In [ ]:
is_direct_sub("trulens_eval.feedback.sadf", "trulens_eval")

In [ ]:
is_direct_sub(qualname(TruLlama), qualname(tru_llama))

In [ ]:
from trulens_eval.tru_llama import TruLlama

In [ ]:
TruLlama.__module__

In [ ]:
from trulens_eval import tru_llama
inspect.getmembers_static(trulens_eval)